In [94]:
import pandas as pd

#读取数据并处理数据
def readcvs(path):
    data=pd.read_csv(path,encoding='gbk')
    data=data.drop_duplicates()               #去重
    columns=['商品规格','选购商品','支付方式', '订单提交时间']
    data[columns] = data[columns].applymap(lambda x: x.replace('\t', ''))
    data.loc[data['流量来源'] == '-', '流量来源'] = '其他'
    return data
def OCR(path):
#有效订单率
    data=readcvs(path)
    x = (data['订单状态'] != '已关闭').sum()
    return f'有效订单总数为{x}\t有效订单率为{x/data.shape[0]*100:.2f}%\n无效订单总数为{data.shape[0]-x}\t无效订单率为{100-x/data.shape[0]*100:.2f}%'
def self_volume(path):
    data = readcvs(path)
    self_volume = data.loc[data['支付完成时间'].notnull(), '订单应付金额'].sum()
    data['支付完成时间'] = pd.to_datetime(data['支付完成时间'])  # 处理缺失值
    self_volume = self_volume  / data['支付完成时间'].dt.date.nunique()
    return f'天均销售额为：{self_volume:.2f}'


def good(path):
    data = readcvs(path)
    goods=data.groupby('商品ID').agg(总订单量=('选购商品','count'), 订单有效数量=('订单状态', lambda x: (x != '已关闭').sum())).sort_values('总订单量',ascending=False)
    goods['有效订单率']=goods['订单有效数量']/goods['总订单量']
    goods['有效订单率']=goods['有效订单率'].apply(lambda x : f'{x*100:.2f}%')
    return goods.head()

#天均销售订单量
def date_self(path):
    data = readcvs(path)
    data['支付完成时间'] = pd.to_datetime(data['支付完成时间'])
    data_self= data['支付完成时间'].isna().count()/data['支付完成时间'].dt.date.nunique()
    return f'天均订单销售量为：{data_self}'



def refund(path):
#总体退款率
    data=readcvs(path)
    refund_num=((data['售后状态'] == '退款成功') | (data['售后状态'] == '待退货') | (data['售后状态'] == '代收退货')).sum()
    x=(data['订单状态']!= '已关闭').sum()
    return  f'总退款数为{refund_num}\t退款率为{refund_num/x*100:.2f}%'

def refund_all(path):
#退款占比
    data=readcvs(path)
    refund=data[(data['售后状态'] == '退款成功') | (data['售后状态'] == '待退货') | (data['售后状态'] == '代收退货')]
    refund=refund.groupby('商品ID').agg(退款数量=('商品ID','count')).sort_values('退款数量',ascending=False)
    refund=pd.concat([good(path)['订单有效数量'],refund['退款数量']],axis=1,sort=False)
    refund['退款率']=refund['退款数量']/refund['订单有效数量']*100
    refund['退款数量'] = refund['退款数量'].fillna(0).replace([np.inf, -np.inf], 0).astype(int)
    refund['退款率']=refund['退款率'].apply(lambda x : f'{x:.2f}%')
    refund[refund['退款率']=='nan%']='0%'
    return refund.head(3)

def pay(path):
#支付方式
    data=readcvs(path)
    pay=data.groupby('支付方式').agg(数量=('支付方式','count')).sort_values('数量',ascending=False)
    pay['占比']=pay['数量']/pay['数量'].sum()*100
    pay['占比']=pay['占比'].apply(lambda x : f'{x:.2f}%')
    return pay.head()


# def size(path):
# #商品规格占比
#     data = readcvs(path)
#     size=data.groupby('商品规格').agg(数量=('商品规格','count')).sort_values(by='数量',ascending=False)
#     size['占比']=size['数量']/size['数量'].sum()*100
#     size['占比']=size['占比'].apply(lambda x : f'{x:.2f}%')
#
#     return size.head()

def area(path):
#地区占比
    data = readcvs(path)
    area=data.groupby('省').agg(数量=('省','count')).sort_values('数量',ascending=False)
    area['占比']=area['数量']/area['数量'].sum()
    area['占比']=area['占比'].apply(lambda x : f'{x*100:.2f}%' )
    return area.head()

def way(path):
#app渠道
    data = readcvs(path)
    way=data.groupby('APP渠道').agg(数量=('APP渠道','count')).sort_values('数量',ascending=False)
    way['占比']=way['数量']/way['数量'].sum()*100
    way['占比']=way['占比'].apply(lambda x : f'{x:.2f}%')
    return way.head()

def try_way(path):
#流量体裁
    data = readcvs(path)
    trf_way=data.groupby('流量体裁').agg(数量=('流量体裁','count')).sort_values('数量',ascending=False)
    try:
        trf_way.drop('数据将于第二天更新',axis=0,inplace=True)
    except:
        pass
    trf_way['占比']=trf_way['数量']/trf_way['数量'].sum()*100
    trf_way['占比']=trf_way['占比'].apply(lambda x : f'{x:.2f}%')
    return trf_way.head()


def source(path):
#流量来源占比
    data = readcvs(path)
    source=data.groupby('流量来源').agg(数量=('流量来源','count')).sort_values('数量',ascending=False)
    source['占比']=source['数量']/source['数量'].sum()*100
    source['占比']=source['占比'].apply(lambda x : f'{x:.2f}%')
    return source.head()


def ad_source(path):
    data=readcvs(path)
    ad_source=data.groupby('广告渠道').agg(数量=('广告渠道','count')).sort_values('数量',ascending=False)
    ad_source['占比']=ad_source['数量']/ad_source['数量'].sum()*100
    ad_source['占比']=ad_source['占比'].apply(lambda x : f'{x:.2f}%')
    return ad_source.head()

def low(path):
#已完成订单中降价占比
    data = readcvs(path)
    low=data[(data['订单状态'] != '已关闭') & (data['降价类优惠']!=0)].shape[0] / data.loc[data['订单状态'] != '已关闭'].shape[0]
    return f'购买降价商品的订单比例：{low*100:.2f}%'



#取消原因
def cancel(path):
    data=readcvs(path)
    return data[data['订单状态']=='已关闭'].groupby('取消原因').agg(数量=('取消原因','count')).sort_values('数量',ascending=False)

def ax_shop(path):
#安心购
    data = readcvs(path)
    ax_shop=(data['是否安心购']=='是').sum()/data.shape[0]*100
    return f'选择安心购的比例为:{ax_shop:.2f}%\n未选择安心购的比例为:{100-ax_shop:.2f}%'

def hour(path):
#时间占比
    data = readcvs(path)
    data.dropna(subset=['支付完成时间'], inplace=True)
    data['支付完成时间']=pd.to_datetime(data['支付完成时间'])
    data['时间段']=data['支付完成时间'].dt.hour.astype(int)
    hour=data.groupby('时间段').agg(订单数量=('时间段','count')).sort_values('订单数量',ascending=False)
    hour['占比']=hour['订单数量']/hour['订单数量'].sum()*100
    hour['占比']=hour['占比'].apply(lambda x : f'{x:.2f}%')
    return hour



def concat(week, half_month, month):
    merged_df = pd.concat([week, half_month, month], axis=1, sort=False, keys=['近7日', '上半个月', '上个月'])
    return merged_df





In [95]:
import pandas as pd


path=['E:/泰迪/资料/数据分析/抖音商城运营-总助数据统计及分析/抖音商城运营-总助数据统计及分析/近7天订单数据.csv',
      'E:/泰迪/资料/数据分析/抖音商城运营-总助数据统计及分析/抖音商城运营-总助数据统计及分析/上半月订单数据.csv',
      'E:/泰迪/资料/数据分析/抖音商城运营-总助数据统计及分析/抖音商城运营-总助数据统计及分析/上月订单数据.csv']
pd.set_option('display.unicode.east_asian_width', True)

print(f'近七日数据\n{date_self(path[0])}\n\n上半个月数据\n{date_self(path[1])}\n\n上个月数据\n{date_self(path[2])}\n')




近七日数据
天均订单销售量为：2134.25

上半个月数据
天均订单销售量为：2104.25

上个月数据
天均订单销售量为：3001.3



In [96]:
#按商品分类，统计三个时间段的数量以及比例
concat(good(path[0]), good(path[1]), good(path[2]))



近7日                         上半个月               \
                    总订单量 订单有效数量 有效订单率 总订单量 订单有效数量   
商品ID                                                                       
3495454119500636656      120           86     71.67%      481          361   
3495456657885023346      483          355     73.50%     2752         1937   
3513300378596824808     2350         1797     76.47%     5408         3941   
3573991726941080892     4883         4130     84.58%    13945        11121   
3595154472139107812     8954         7143     79.77%    10237         7660   

                                 上个月                          
                    有效订单率 总订单量 订单有效数量 有效订单率  
商品ID                                                           
3495454119500636656     75.05%     1844         1442     78.20%  
3495456657885023346     70.39%     7188         5132     71.40%  
3513300378596824808     72.87%    18384        13406     72.92%  
3573991726941080892     79.75%    25717        20637     80.25%  
3595154472139107812     74.83%    35265        26708     75.74%

In [97]:
#统计有效订单率
print(f'近七日数据\n{OCR(path[0])}\n\n上半个月数据\n{OCR(path[1])}\n\n上个月数据\n{OCR(path[2])}\n')



近七日数据
有效订单总数为13722	有效订单率为80.37%
无效订单总数为3352	无效订单率为19.63%

上半个月数据
有效订单总数为25640	有效订单率为76.16%
无效订单总数为8028	无效订单率为23.84%

上个月数据
有效订单总数为68603	有效订单率为76.19%
无效订单总数为21436	无效订单率为23.81%



In [98]:
#三个时间段的退款数量以及其占比
concat(refund_all(path[0]),refund_all(path[1]),refund_all(path[2]))



近7日                      上半个月           \
                    订单有效数量 退款数量  退款率 订单有效数量 退款数量   
商品ID                                                                    
3495454119500636656           86        9  10.47%          361       38   
3495456657885023346          355       40  11.27%         1937      254   
3513300378596824808         1797      226  12.58%         3941      710   

                                  上个月                   
                     退款率 订单有效数量 退款数量  退款率  
商品ID                                                     
3495454119500636656  10.53%         1442      118   8.18%  
3495456657885023346  13.11%         5132      668  13.02%  
3513300378596824808  18.02%        13406     1953  14.57%

In [99]:
#三个时间段的支付方式数量以及占比
concat(pay(path[0]),pay(path[1]),pay(path[2]))



近7日         上半个月         上个月        
          数量    占比     数量    占比   数量    占比
抖音支付  5056  29.61%     9606  28.53%  25044  27.81%
支付宝    4826  28.27%     9232  27.42%  22979  25.52%
抖音月付  4768  27.93%     9279  27.56%  25807  28.66%
微信      2363  13.84%     5442  16.16%  16016  17.79%
极速付      61   0.36%      109   0.32%    191   0.21%

In [100]:
#三个时间段的地区占比
concat(area(path[0]),area(path[1]),area(path[2]))



近7日         上半个月         上个月        
        数量    占比     数量    占比   数量    占比
省                                                  
广东省  1802  10.55%     4130  12.27%  10732  11.92%
江苏省  1744  10.21%     3263   9.69%   8895   9.88%
浙江省  1570   9.20%     3189   9.47%   8589   9.54%
山东省  1018   5.96%     1924   5.71%   5038   5.60%
四川省   819   4.80%     1706   5.07%   4322   4.80%

In [101]:
#app渠道
concat(way(path[0]),way(path[1]),way(path[2]))



近7日         上半个月         上个月        
                    数量    占比     数量    占比   数量    占比
抖音               11620  68.06%    24133  71.68%  67881  75.39%
抖音极速版          2621  15.35%     5460  16.22%  13026  14.47%
头条-抖音电商       1203   7.05%     1179   3.50%   2514   2.79%
番茄小说-抖音电商    401   2.35%      619   1.84%    888   0.99%
今日头条             392   2.30%      924   2.74%   2545   2.83%

In [102]:
#流量体裁
concat(try_way(path[0]),try_way(path[1]),try_way(path[2]))



近7日         上半个月         上个月        
        数量    占比     数量    占比   数量    占比
直播    8287  52.99%    11867  35.25%  35815  39.78%
短视频  6038  38.61%    20293  60.27%  50927  56.56%
其他     883   5.65%      440   1.31%    876   0.97%
商品卡   430   2.75%     1068   3.17%   2421   2.69%

In [103]:
#流量来源
concat(source(path[0]),source(path[1]),source(path[2]))



近7日         上半个月         上个月        
           数量    占比     数量    占比   数量    占比
流量来源                                               
精选联盟  14287  83.68%    20958  62.25%  60257  66.92%
其他       2453  14.37%    11897  35.34%  28012  31.11%
小店自卖    334   1.96%      813   2.41%   1770   1.97%

In [104]:
#购买降价商品的订单比例
print(f'近七日数据\n{low(path[0])}\n\n上半个月数据\n{low(path[1])}\n\n上个月数据\n{low(path[2])}\n')


近七日数据
购买降价商品的订单比例：51.68%

上半个月数据
购买降价商品的订单比例：29.63%

上个月数据
购买降价商品的订单比例：6.65%



In [105]:

#取消原因以及比例
# concat(cancel(path[0]),cancel(path[1]),cancel(path[2]))



In [106]:
#选择安心购
print(f'近七日数据\n{ax_shop(path[0])}\n\n上半个月数据\n{ax_shop(path[1])}\n\n上个月数据\n{ax_shop(path[2])}\n')



近七日数据
选择安心购的比例为:9.76%
未选择安心购的比例为:90.24%

上半个月数据
选择安心购的比例为:11.76%
未选择安心购的比例为:88.24%

上个月数据
选择安心购的比例为:28.11%
未选择安心购的比例为:71.89%



In [107]:
#各时间段比例
concat(hour(path[0]),hour(path[1]),hour(path[2]))

近7日        上半个月          上个月       
       订单数量   占比 订单数量   占比 订单数量   占比
时间段                                                
0           355  2.38%      926  3.22%     2464  3.20%
1           286  1.92%      783  2.72%     1736  2.26%
2           193  1.30%      572  1.99%     1220  1.59%
3           136  0.91%      378  1.31%      921  1.20%
4           125  0.84%      354  1.23%      988  1.28%
5           254  1.71%      737  2.56%     1674  2.18%
6           309  2.08%      995  3.46%     2818  3.66%
7           308  2.07%     1116  3.88%     2618  3.40%
8           585  3.93%      994  3.45%     2406  3.13%
9          1252  8.41%      965  3.35%     2119  2.75%
10         1428  9.59%     1140  3.96%     2815  3.66%
11         1125  7.56%     1249  4.34%     5771  7.50%
12         1314  8.83%     2234  7.76%     6706  8.72%
13         1289  8.66%     2153  7.48%     4311  5.60%
14          882  5.92%     1324  4.60%     4940  6.42%
15         1242  8.34%     1939  6.74%     2630  3.42%
16          893  6.00%     1376  4.78%     2094  2.72%
17          698  4.69%     1226  4.26%     4154  5.40%
18          296  1.99%      965  3.35%     3068  3.99%
19          281  1.89%     1485  5.16%     5402  7.02%
20          334  2.24%     1994  6.93%     6157  8.00%
21          418  2.81%     1562  5.43%     4184  5.44%
22          417  2.80%     1155  4.01%     2912  3.79%
23          469  3.15%     1164  4.04%     2821  3.67%

In [110]:
print(f'近七日数据\n{self_volume(path[0])}\n\n上半个月数据\n{self_volume(path[1])}\n\n上个月数据\n{self_volume(path[2])}\n')

近七日数据
天均销售额为：117297.46

上半个月数据
天均销售额为：113749.13

上个月数据
天均销售额为：164027.16

